In [72]:
import requests
import time
import re
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [73]:
class law_presidencia:
    def __init__(self):  
        self.titulo = ""
        self.data_assinatura = ""
        self.ementa = ""
        self.situacao = ""
        self.chefe_governo = ""
        self.origem = ""
        self.data_publicacao = ""
        self.fonte = ""
        self.fonte_link = ""
        self.texto_integral = ""
        self.referenda = ""
        self.alteracao = ""
        self.correlacao = ""
        self.veto = ""
        self.assunto = ""
        self.classificacao_direito = ""
        self.obs = ""
        
    def print_self(self):
        return(self.titulo + "*" + self.data_assinatura + "*" + self.ementa + "*" + self.situacao + "*" \
              + self.chefe_governo + "*" + self.origem + "*" + self.data_publicacao + "*" \
              + self.fonte + "*" + self.fonte_link + "*" + self.texto_integral + "*" \
              + self.referenda + "*" + self.alteracao + "*" + self.correlacao + "*" \
              + self.veto + "*" + self.assunto + "*" + self.classificacao_direito + "*" \
              + self.obs + "\n")

In [74]:
dictionary = {
    "Data de assinatura:":       "data_assinatura",
    "Ementa:":                   "ementa",
    "Situação:":                 "situacao",
    "Chefe de Governo:":         "chefe_governo",
    "Origem:":                   "origem",
    "Data de Publicação:":       "data_publicacao",
    "Referenda:":                "referenda",
    "Alteração:":                "alteracao",
    "Correlação:":               "correlacao", 
    "Veto:":                     "veto", 
    "Assunto:":                  "assunto", 
    "Classificação de direito:": "classificacao_direito", 
    "Observação:":               "obs"
}

In [75]:
laws = []
laws_gone_wrong = []

In [76]:
options = uc.ChromeOptions()
options.add_argument('ignore-certificate-errors')
driver = uc.Chrome(chrome_options = options)
req = driver.get("https://legislacao.presidencia.gov.br/atos/?tipo=DEL&numero=547&ano=1969&data=18/04/1969&ato=cb9QzYU5UejRVTc05")

In [77]:
def scrape_page_presidencia(url):
    driver.get(url)
    element_present = EC.presence_of_element_located((By.ID, 'barra-brasil'))
    WebDriverWait(driver, 10).until(element_present)

    current_law = law_presidencia()
    soup = BeautifulSoup(driver.page_source, "html.parser")

    tags = soup.findAll('h2')
    
    time.sleep(0.5)
    
    current_law.titulo = soup.find('h1').text.strip()

    for tag in tags:
        tag_text = tag.text.strip()
        value = tag.find_next().text.strip()
        value = re.sub(' +', ' ', value)
        value = value.replace('\n', '').replace('\r', '').capitalize()
        
        if tag_text == "Link:":
            current_law.texto_integral = tag.find_next().find('a')['href']
        elif tag_text == "Fonte:":
            current_law.fonte = value
            try:
                current_law.fonte_link = tag.find_next().find('a')['href']
            except:
                current_law.fonte_link = ""
        
        elif tag_text in dictionary:
            if(value == "---"):
                value = ""
            setattr(current_law, dictionary[tag_text], value)
    return current_law

In [78]:
file = open("links_leis_federais_planalto.csv", "r")

lines = file.readlines()[11600:]

i =0
for line in lines:
    url = line.split(";")[1]
    laws.append(scrape_page_presidencia(url))
    i += 1
    if (i % 100 == 0):
        print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700


In [79]:
saida = open("saida_socorro.txt", "w")
saida.write("nome*data assinatura*ementa*situacao*chefe_governo*origem*data_publicacao" +\
           "*fonte*fonte_link*texto integral*referenda*alteracao*correlacao*veto" +\
           "*assunto*cassificacao direito*observacao*")
for l in laws:
    saida.write(l.titulo + "*" + l.data_assinatura + "*" + l.ementa + "*" + l.situacao + "*" \
              + l.chefe_governo + "*" + l.origem + "*" + l.data_publicacao + "*" \
              + l.fonte + "*" + l.fonte_link + "*" + l.texto_integral + "*" \
              + l.referenda + "*" + l.alteracao + "*" + l.correlacao + "*" \
              + l.veto + "*" + l.assunto + "*" + l.classificacao_direito + "*" \
              + l.obs + "\n")